# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [96]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df = pd.read_csv("assets/nhl.csv")
cities = pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]] 

#print(cities)
#Readability 
cities.rename(columns={'Population (2016 est.)[8]': 'Population'}, inplace=True)

#remove the '[NoteX]' values
# Remove '[NoteX]' values only for the 'MLB' column
cities['NHL'] = cities['NHL'].str.replace(r'\[.*\]', '', regex=True)

# Print the cities DataFrame to verify changes
print(cities)


#Only relevant teams
print(nhl_df)
#Lets drop invalid rows (Division values + only 2018)
nhl_df_filtered = nhl_df[:35].drop([0, 9, 18, 26]).reset_index(drop=True)
print('before:',len(nhl_df), 'after:', len(nhl_df_filtered))
#print(nhl_df_filtered)
#Remove '*' from certain team names
nhl_df_filtered['team'] = nhl_df_filtered['team'].str.replace(r'\*', '', regex=True)
#print(nhl_df_filtered)

team_to_region = pd.Series({
    0: 'Tampa Bay Area',
    1: 'Boston',
    2: 'Toronto',
    3: 'Miami–Fort Lauderdale',
    4: 'Detroit',
    5: 'Montreal',
    6: 'Ottawa',
    7: 'Buffalo',
    8: 'Washington, D.C.',
    9: 'Pittsburgh',
    10: 'Philadelphia',
    11: 'Columbus',
    12: 'New York City',
    13: 'Raleigh',
    14: 'New York City',
    15: 'New York City',
    16: 'Nashville',
    17: 'Winnipeg',
    18: 'Minneapolis–Saint Paul',
    19: 'Denver',
    20: 'St. Louis',
    21: 'Dallas–Fort Worth',
    22: 'Chicago',
    23: 'Las Vegas',
    24: 'Los Angeles',
    25: 'San Francisco Bay Area',
    26: 'Los Angeles',
    27: 'Calgary',
    28: 'Edmonton',
    29: 'Vancouver',
    30: 'Phoenix',
}, name='Metropolitan area')    

def nhl_correlation():
    nhl_df_filtered['Metropolitan area'] = team_to_region
    city_population = cities[['Metropolitan area', 'Population']]
    nhl_with_population = pd.merge(nhl_df_filtered, city_population, on='Metropolitan area')
    
    nhl_with_population['W'] = nhl_with_population['W'].astype(float)
    nhl_with_population['L'] = nhl_with_population['L'].astype(float)
    nhl_with_population['Population'] = nhl_with_population['Population'].astype(int)

    #Calculate w/l ratio
    nhl_with_population['W/L ratio'] = nhl_with_population['W'] / (nhl_with_population['W'] + nhl_with_population['L'])

    population_by_region = nhl_with_population.groupby('Metropolitan area')['Population'].first()

    #mean average for regions with > 1 teams
    win_loss_by_region = nhl_with_population.groupby('Metropolitan area')['W/L ratio'].mean()

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    
    correlation, p_value= stats.pearsonr(population_by_region, win_loss_by_region)
    return correlation


nhl_correlation()

         Metropolitan area Population                    NFL  \
0            New York City   20153634    Giants Jets[note 1]   
1              Los Angeles   13310447  Rams Chargers[note 4]   
2   San Francisco Bay Area    6657982  49ers Raiders[note 6]   
3                  Chicago    9512999          Bears[note 8]   
4        Dallas–Fort Worth    7233323                Cowboys   
5         Washington, D.C.    6131977               Redskins   
6             Philadelphia    6070500                 Eagles   
7                   Boston    4794447      Patriots[note 14]   
8   Minneapolis–Saint Paul    3551036                Vikings   
9                   Denver    2853077                Broncos   
10   Miami–Fort Lauderdale    6066387               Dolphins   
11                 Phoenix    4661537              Cardinals   
12                 Detroit    4297617                  Lions   
13                 Toronto    5928040            — [note 22]   
14                 Houston    6772470   

0.012486162921209909

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [40]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

cities.rename(columns={'Population (2016 est.)[8]': 'Population'}, inplace=True)
#lets remove the '[NoteX]' values
cities['NBA'] = cities['NBA'].str.replace(r'\[.*\]', '', regex=True)

nba_df_filtered = nba_df[nba_df['year'] == 2018]
print(nba_df_filtered.head(30))
nba_df_filtered['team'] = nba_df_filtered['team'].str.replace('\s\(\d*\)' ,'',regex=True)

#Lets drop invalid rows (Division values + only 2018)
#nba_df_filtered = nba_df[:35].drop([0, 9, 18, 26]).reset_index(drop=True)
print('before:',len(nba_df), 'after:', len(nba_df_filtered))
#Remove '*' from certain team names
nba_df_filtered['team'] = nba_df_filtered['team'].str.replace(r'\*', '', regex=True)
#print(nba_df_filtered)

team_to_region = pd.Series({
    0:'Toronto',
    1:'Boston',
    2:'Philadelphia',
    3:'Cleveland',
    4:'Indianapolis',
    5:'Miami–Fort Lauderdale',
    6:'Milwaukee',
    7:'Washington, D.C.',
    8:'Detroit',
    9:'Charlotte',
    10:'New York City',
    11:'New York City',
    12:'Chicago',
    13:'Orlando',
    14:'Atlanta',
    15:'Houston',
    16:'San Francisco Bay Area',
    17:'Portland',
    18:'Oklahoma City',
    19:'Salt Lake City',
    20:'New Orleans',
    21:'San Antonio',
    22:'Green Bay',
    23:'Denver',
    24:'Los Angeles',
    25:'Los Angeles',
    26:'Sacramento',
    27:'Dallas–Fort Worth',
    28:'Memphis',
    29:'Phoenix',
})

def nba_correlation():
    #Dodac kolumne metropolitan area do glownego df
    nba_df_filtered['Metropolitan area'] = team_to_region
    # tuple dla city_population
    city_population = cities[['Metropolitan area', 'Population']]
    #merge city population z df
    nba_with_population = pd.merge(nba_df_filtered, city_population, on ='Metropolitan area')
    
    #w/l preparation
    nba_with_population['W'] = nba_with_population['W'].astype(float)
    nba_with_population['L'] = nba_with_population['L'].astype(float)
    nba_with_population['Population'] = nba_with_population['Population'].astype(int)

    #wl ratio
    nba_with_population['W/L ratio'] = nba_with_population['W'] / (nba_with_population['W'] +  nba_with_population['L'])
    print(nba_with_population.head(30))
    population_by_region = nba_with_population.groupby('Metropolitan area')['Population'].first()
    win_loss_by_region = nba_with_population.groupby('Metropolitan area')['W/L ratio'].mean()
    
    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    correlation, p_value = stats.pearsonr(population_by_region, win_loss_by_region)
    return correlation

nba_correlation()

                           team   W   L                 W/L%    GB   PS/G  \
0          Toronto Raptors* (1)  59  23                 0.72     —  111.7   
1           Boston Celtics* (2)  55  27   0.6709999999999999   4.0  104.0   
2       Philadelphia 76ers* (3)  52  30                0.634   7.0  109.8   
3      Cleveland Cavaliers* (4)  50  32                 0.61   9.0  110.9   
4           Indiana Pacers* (5)  48  34                0.585  11.0  105.6   
5               Miami Heat* (6)  44  38                0.537  15.0  103.4   
6          Milwaukee Bucks* (7)  44  38                0.537  15.0  106.5   
7       Washington Wizards* (8)  43  39                0.524  16.0  106.6   
8           Detroit Pistons (9)  39  43  0.47600000000000003  20.0  103.8   
9        Charlotte Hornets (10)  36  46                0.439  23.0  108.2   
10         New York Knicks (11)  29  53                0.354  30.0  104.5   
11           Brooklyn Nets (12)  28  54                0.341  31.0  106.6   

-0.18564814421159903

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [63]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

cities['MLB'] = cities['MLB'].str.replace(r'\[.*\]', '', regex=True)
cities.rename(columns={'Population (2016 est.)[8]':'Population'}, inplace=True)
mlb_df_filtered = mlb_df[:30]
mlb_df_filtered['team'] = mlb_df_filtered['team'].str.replace('\s\(\d*\)' ,'',regex=True)

#print(mlb_df_filtered.head(30))
#mlb_df['
#print(cities)
mlb_series = pd.Series({
    0:'Boston',
    1:'New York City',
    2:'Tampa Bay Area',
    3:'Toronto',
    4:'Baltimore',
    5:'Cleveland',
    6:'Minneapolis–Saint Paul',
    7:'Detroit',
    8:'Chicago',
    9:'Kansas City',
    10:'Houston',
    11:'San Francisco Bay Area',
    12:'Seattle',
    13:'Los Angeles',
    14:'Dallas–Fort Worth',
    15:'Atlanta',
    16:'Washington, D.C.',
    17:'Philadelphia',
    18:'New York City',
    19:'Miami–Fort Lauderdale',
    20:'Milwaukee',
    21:'Chicago',
    22:'St. Louis',
    23:'Pittsburgh',
    24:'Cincinnati',
    25:'Los Angeles',
    26:'Denver',
    27:'Phoenix',
    28:'San Francisco Bay Area',
    29:'San Diego',
})

def mlb_correlation(): 
    mlb_df_filtered['Metropolitan area'] = mlb_series
    city_population = cities[['Metropolitan area', 'Population']]
    mlb_df_filtered2 = pd.merge(mlb_df_filtered, city_population, on = 'Metropolitan area')

    mlb_df_filtered2['W'] = mlb_df_filtered2['W'].astype(float)
    mlb_df_filtered2['L'] = mlb_df_filtered2['L'].astype(float)
    mlb_df_filtered2['Population'] = mlb_df_filtered2['Population'].astype(int)
    
    mlb_df_filtered2['W/L ratio'] = mlb_df_filtered2['W'] / (mlb_df_filtered2['W'] + mlb_df_filtered2['L'])
    print(mlb_df_filtered2.head())
    
    
    population_by_region = mlb_df_filtered2.groupby('Metropolitan area')['Population'].first()
    win_loss_by_region = mlb_df_filtered2.groupby('Metropolitan area')['W/L ratio'].mean()
    

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"
    
    correlation, p_value = stats.pearsonr(population_by_region, win_loss_by_region)
    return correlation
mlb_correlation()

                team      W     L   W-L%    GB  year League Metropolitan area  \
0     Boston Red Sox  108.0  54.0  0.667    --  2018    MLB            Boston   
1   New York Yankees  100.0  62.0  0.617   8.0  2018    MLB     New York City   
2      New York Mets   77.0  85.0  0.475  13.0  2018    MLB     New York City   
3     Tampa Bay Rays   90.0  72.0  0.556  18.0  2018    MLB    Tampa Bay Area   
4  Toronto Blue Jays   73.0  89.0  0.451  35.0  2018    MLB           Toronto   

   Population  W/L ratio  
0     4794447   0.666667  
1    20153634   0.617284  
2    20153634   0.475309  
3     3032171   0.555556  
4     5928040   0.450617  


0.15027698302669307

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [91]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df = pd.read_csv("assets/nfl.csv")
cities = pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]

cities.rename(columns={'Population (2016 est.)[8]': 'Population'}, inplace=True)
cities['NFL'] = cities['NFL'].str.replace(r'\[.*\]', '', regex=True)

nfl_series = pd.Series({
    0: 'Boston',
    1: 'Miami–Fort Lauderdale',
    2: 'Buffalo',
    3: 'New York City',
    4: 'Baltimore',
    5: 'Pittsburgh',
    6: 'Cleveland',
    7: 'Cincinnati',
    8: 'Houston',
    9: 'Indianapolis',
    10: 'Nashville',
    11: 'Jacksonville',
    12: 'Kansas City',
    13: 'Los Angeles',
    14: 'Denver',
    15: 'San Francisco Bay Area',
    16: 'Dallas–Fort Worth',
    17: 'Philadelphia',
    18: 'Washington, D.C.',
    19: 'New York City',
    20: 'Chicago',
    21: 'Minneapolis–Saint Paul',
    22: 'Green Bay',
    23: 'Detroit',
    24: 'New Orleans',
    25: 'Charlotte',
    26: 'Atlanta',
    27: 'Tampa Bay Area',
    28: 'Los Angeles',
    29: 'Seattle',
    30: 'San Francisco Bay Area',
    31: 'Phoenix',
}, name='Metropolitan area')

def nfl_correlation():
    nfl_df_filtered = nfl_df[nfl_df['year'] == 2018].reset_index(drop=True)
    nfl_df_filtered = nfl_df_filtered.iloc[:40]  # Filter down to 40 rows
    nfl_df_filtered = nfl_df_filtered.drop([0, 5, 10, 15, 20, 25, 30, 35]).reset_index(drop=True)
    nfl_df_filtered['Metropolitan area'] = nfl_series

    new_cities4 = cities[['Metropolitan area', 'Population']]
    new_nfl = pd.merge(nfl_df_filtered, new_cities4, on='Metropolitan area')

    new_nfl['W'] = new_nfl['W'].astype(float)
    new_nfl['L'] = new_nfl['L'].astype(float)
    new_nfl['Population'] = new_nfl['Population'].astype(int)
    new_nfl['W-L%'] = new_nfl['W-L%'].astype(float)

    population_by_region = new_nfl.groupby('Metropolitan area')['Population'].first()
    win_loss_by_region = new_nfl.groupby('Metropolitan area')['W-L%'].mean()

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"
    correlation, p_value = stats.pearsonr(population_by_region, win_loss_by_region)
    return correlation


nfl_correlation()

0.004282141436393022

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [105]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df = pd.read_csv("assets/mlb.csv")
nhl_df = pd.read_csv("assets/nhl.csv")
nba_df = pd.read_csv("assets/nba.csv")
nfl_df = pd.read_csv("assets/nfl.csv")
cities = pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]

cities.rename(columns={'Population (2016 est.)[8]': 'Population'}, inplace=True)
for sport in ['NFL', 'MLB', 'NBA', 'NHL']:
    cities[sport] = cities[sport].str.replace(r'\[.*\]', '', regex=True)

# Clean the team names
def clean_team_names(df):
    df['team'] = df['team'].str.replace(r'\*', '', regex=True)
    df['team'] = df['team'].str.replace(r'\+', '', regex=True)
    df['team'] = df['team'].str.strip()
    return df

mlb_df = clean_team_names(mlb_df)
nhl_df = clean_team_names(nhl_df)
nba_df = clean_team_names(nba_df)
nfl_df = clean_team_names(nfl_df)

mlb_df['W'] = pd.to_numeric(mlb_df['W'], errors='coerce')
mlb_df['L'] = pd.to_numeric(mlb_df['L'], errors='coerce')
nhl_df['W'] = pd.to_numeric(nhl_df['W'], errors='coerce')
nhl_df['L'] = pd.to_numeric(nhl_df['L'], errors='coerce')
nba_df['W'] = pd.to_numeric(nba_df['W'], errors='coerce')
nba_df['L'] = pd.to_numeric(nba_df['L'], errors='coerce')
nfl_df['W'] = pd.to_numeric(nfl_df['W'], errors='coerce')
nfl_df['L'] = pd.to_numeric(nfl_df['L'], errors='coerce')

mlb_df = mlb_df[mlb_df['year'] == 2018]
nhl_df = nhl_df[nhl_df['year'] == 2018]
nba_df = nba_df[nba_df['year'] == 2018]
nfl_df = nfl_df[nfl_df['year'] == 2018]

team_to_region = {
    "Boston Red Sox": "Boston",
    "New York Yankees": "New York City",
    "Toronto Blue Jays": "Toronto",
    "Baltimore Orioles": "Baltimore",
    "Tampa Bay Rays": "Tampa Bay Area",
    "Chicago White Sox": "Chicago",
    "Cleveland Indians": "Cleveland",
    "Detroit Tigers": "Detroit",
    "Kansas City Royals": "Kansas City",
    "Minnesota Twins": "Minneapolis–Saint Paul",
    "Houston Astros": "Houston",
    "Los Angeles Angels": "Los Angeles",
    "Oakland Athletics": "San Francisco Bay Area",
    "Seattle Mariners": "Seattle",
    "Texas Rangers": "Dallas–Fort Worth",
    "Atlanta Braves": "Atlanta",
    "Miami Marlins": "Miami–Fort Lauderdale",
    "New York Mets": "New York City",
    "Philadelphia Phillies": "Philadelphia",
    "Washington Nationals": "Washington, D.C.",
    "Chicago Cubs": "Chicago",
    "Cincinnati Reds": "Cincinnati",
    "Milwaukee Brewers": "Milwaukee",
    "Pittsburgh Pirates": "Pittsburgh",
    "St. Louis Cardinals": "St. Louis",
    "Arizona Diamondbacks": "Phoenix",
    "Colorado Rockies": "Denver",
    "Los Angeles Dodgers": "Los Angeles",
    "San Diego Padres": "San Diego",
    "San Francisco Giants": "San Francisco Bay Area",
    "Boston Bruins": "Boston",
    "New York Rangers": "New York City",
    "Toronto Maple Leafs": "Toronto",
    "Washington Capitals": "Washington, D.C.",
    "Philadelphia Flyers": "Philadelphia",
    "Pittsburgh Penguins": "Pittsburgh",
    "Chicago Blackhawks": "Chicago",
    "Detroit Red Wings": "Detroit",
    "St. Louis Blues": "St. Louis",
    "Los Angeles Kings": "Los Angeles",
    "Dallas Stars": "Dallas–Fort Worth",
    "Minnesota Wild": "Minneapolis–Saint Paul",
    "Tampa Bay Lightning": "Tampa Bay Area",
    "San Jose Sharks": "San Francisco Bay Area",
    "Boston Celtics": "Boston",
    "Brooklyn Nets": "New York City",
    "New York Knicks": "New York City",
    "Toronto Raptors": "Toronto",
    "Philadelphia 76ers": "Philadelphia",
    "Chicago Bulls": "Chicago",
    "Cleveland Cavaliers": "Cleveland",
    "Detroit Pistons": "Detroit",
    "Indiana Pacers": "Indianapolis",
    "Milwaukee Bucks": "Milwaukee",
    "Miami Heat": "Miami–Fort Lauderdale",
    "Atlanta Hawks": "Atlanta",
    "Washington Wizards": "Washington, D.C.",
    "Dallas Mavericks": "Dallas–Fort Worth",
    "Houston Rockets": "Houston",
    "Denver Nuggets": "Denver",
    "Los Angeles Lakers": "Los Angeles",
    "Los Angeles Clippers": "Los Angeles",
    "Golden State Warriors": "San Francisco Bay Area",
    "Phoenix Suns": "Phoenix",
    "Buffalo Bills": "Buffalo",
    "Miami Dolphins": "Miami–Fort Lauderdale",
    "New England Patriots": "Boston",
    "New York Jets": "New York City",
    "Baltimore Ravens": "Baltimore",
    "Cincinnati Bengals": "Cincinnati",
    "Cleveland Browns": "Cleveland",
    "Pittsburgh Steelers": "Pittsburgh",
    "Houston Texans": "Houston",
    "Indianapolis Colts": "Indianapolis",
    "Jacksonville Jaguars": "Jacksonville",
    "Tennessee Titans": "Nashville",
    "Denver Broncos": "Denver",
    "Kansas City Chiefs": "Kansas City",
    "Los Angeles Chargers": "Los Angeles",
    "Oakland Raiders": "San Francisco Bay Area",
    "Dallas Cowboys": "Dallas–Fort Worth",
    "New York Giants": "New York City",
    "Philadelphia Eagles": "Philadelphia",
    "Washington Redskins": "Washington, D.C.",
    "Chicago Bears": "Chicago",
    "Detroit Lions": "Detroit",
    "Green Bay Packers": "Green Bay",
    "Minnesota Vikings": "Minneapolis–Saint Paul",
    "Atlanta Falcons": "Atlanta",
    "Carolina Panthers": "Charlotte",
    "New Orleans Saints": "New Orleans",
    "Tampa Bay Buccaneers": "Tampa Bay Area",
    "Arizona Cardinals": "Phoenix",
    "Los Angeles Rams": "Los Angeles",
    "San Francisco 49ers": "San Francisco Bay Area",
    "Seattle Seahawks": "Seattle"
}

mlb_df['W/L Ratio'] = mlb_df['W'] / (mlb_df['W'] + mlb_df['L'])
nhl_df['W/L Ratio'] = nhl_df['W'] / (nhl_df['W'] + nhl_df['L'])
nba_df['W/L Ratio'] = nba_df['W'] / (nba_df['W'] + nba_df['L'])
nfl_df['W/L Ratio'] = nfl_df['W'] / (nfl_df['W'] + nfl_df['L'])

mlb_df['Metropolitan area'] = mlb_df['team'].map(team_to_region)
nhl_df['Metropolitan area'] = nhl_df['team'].map(team_to_region)
nba_df['Metropolitan area'] = nba_df['team'].map(team_to_region)
nfl_df['Metropolitan area'] = nfl_df['team'].map(team_to_region)

mlb_grouped = mlb_df.groupby('Metropolitan area')['W/L Ratio'].mean()
nhl_grouped = nhl_df.groupby('Metropolitan area')['W/L Ratio'].mean()
nba_grouped = nba_df.groupby('Metropolitan area')['W/L Ratio'].mean()
nfl_grouped = nfl_df.groupby('Metropolitan area')['W/L Ratio'].mean()

def sports_team_performance():
    common_cities = list(set(mlb_grouped.index) & set(nhl_grouped.index) & set(nba_grouped.index) & set(nfl_grouped.index))
    print("Common Cities:", common_cities)

    merged_df = pd.DataFrame({
        'MLB': mlb_grouped,
        'NHL': nhl_grouped,
        'NBA': nba_grouped,
        'NFL': nfl_grouped
    }).loc[common_cities]

    print(merged_df)

    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame(np.nan, index=sports, columns=sports)

    for sport1 in sports:
        for sport2 in sports:
            if sport1 != sport2:
                t_stat, p_val = stats.ttest_rel(merged_df[sport1], merged_df[sport2])
                p_values.loc[sport1, sport2] = p_val

    return p_values

p_values = sports_team_performance()
print(p_values)


Common Cities: []
Empty DataFrame
Columns: [MLB, NHL, NBA, NFL]
Index: []
     NFL  NBA  NHL  MLB
NFL  NaN  NaN  NaN  NaN
NBA  NaN  NaN  NaN  NaN
NHL  NaN  NaN  NaN  NaN
MLB  NaN  NaN  NaN  NaN
